### Definición de la función objetivo y su gradiente:

In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import json
from scipy.optimize import minimize

def f(x):
    return np.arctan(x[0]**2 + x[1]**2) / np.exp(x[0]) 

# Gradiente:
def grad_f(x):
    df_dx = ((2*x[0] / (1 + (x[0]**2 + x[1]**2)**2)) - np.arctan(x[0]**2 + x[1]**2)) / np.exp(x[0])
    df_dy = 2 * x[1] / ((1 + (x[0]**2 + x[1]**2)**2) * np.exp(x[0]))
    return np.array([df_dx, df_dy])

### 🔹Implementación de Método de Máximo Descenso:
⚙️ Parámetros de entrada:

- x0 → punto inicial (vector donde comienza la búsqueda).

- grad_f → gradiente (∇f(x)) de la función objetivo.

- learning_rate → tamaño de paso.

- tol → tolerancia (si el cambio entre iteraciones es menor que este valor, se detiene el algoritmo).

- max_iter → número máximo de iteraciones permitidas.

⚙️ Resultados que devuelve:

- x_min → el último punto calculado (aproximación del mínimo de la función objetivo).

- history → lista con todos los puntos visitados.

- iteraciones → número de pasos realizados.

- succes → True si se detuvo por tolerancia, False si se agotaron las iteraciones.

- time → cuánto tardó en ejecutarse el algoritmo.

In [2]:
def gradient_descent(x0, grad_f, learning_rate=0.1, tol=1e-6, max_iter=100):
    x = np.array(x0, dtype=float)
    history = [x.copy()]
    start = time.time()  

    for i in range(max_iter):
        grad = grad_f(x)
        
        x_new = x - learning_rate * grad
        history.append(x_new.copy())
        
        if np.linalg.norm(x_new - x) < tol:
            end = time.time()
            return x_new, history, i+1, True, end - start
        
        x = x_new

    end = time.time()
    
    return x, history, max_iter, False, end - start

### 🔹 Implementación del método Quasi-Newton (BFGS)

Elementos utilizados:

🔸 Clase `Callback`

Se utiliza para guardar los puntos intermedios (iteraciones) del algoritmo.

🔸función `minimize` de la biblioteca `scipy.optimize` 

Se utiliza para aplicar el **método BFGS (Broyden–Fletcher–Goldfarb–Shanno)**, la técnica de optimización **Quasi-Newton** explicada en el Informe Teórico

---

⚙️ Valores de entrada

- f → Función objetivo a minimizar.  
- x0 → Vector con el punto inicial de la búsqueda.  
- method='BFGS' → Especifica que se usará el método Quasi-Newton BFGS.  
- jac=grad_f → Indica la función que calcula el gradiente de `f`.  
- callback=callback → Función auxiliar que almacena los puntos visitados durante la optimización.  
- options → Diccionario con parámetros de configuración del método:
  - `'gtol': 1e-6` → Criterio de parada basado en la norma del gradiente.  
  - `'disp': False` → Desactiva la impresión de resultados en consola.  
  - `'maxiter': 100` → Número máximo de iteraciones permitidas.

---
⚙️ Resultados que devuelve

- x_qn → Vector con el punto final del proceso (aproximación del mínimo).

- hist_qn → Lista con todos los puntos visitados durante la optimización (trayectoria).

- iterations → número de pasos realizados.

- succes → True si se detuvo por tolerancia, False si se agotaron las iteraciones.

- time_qn → Tiempo total de ejecución del método.

In [3]:
def quasi_newton_method(f, grad_f, x0, tol=1e-6, max_iter=100):
    class Callback:
            def __init__(self):
                self.history = []
            def __call__(self, xk):
                self.history.append(np.array(xk))

    callback = Callback()

    start = time.time()

    res = minimize(f, x0, method='BFGS', jac=grad_f, callback=callback,
        options={'gtol': tol, 'disp': False, 'maxiter': max_iter})
    
    elapsed_time = time.time() - start

    # Results
    x_min = res.x
    history = [np.array(x0)] + callback.history
    iterations = res.nit
    success = res.success

    return x_min, history, iterations, success, elapsed_time

### Funcion para graficar los resultados:

### Función `Experiment`

La función `Experiment` permite automatizar la ejecución de experimentos de optimización para distintos parámetros iniciales y configuraciones. Su funcionamiento resumido es el siguiente:

1. **Lectura de configuraciones:** Carga un archivo JSON (`experiment_name`) que contiene los parámetros de cada experimento, como punto inicial, tolerancia, tasa de aprendizaje y número máximo de iteraciones.

2. **Ejecución de métodos de optimización:**  
   - Aplica el **Método de Máximo Descenso** (`gradient_descent`) con los parámetros especificados.  
   - Aplica el **Método Quasi-Newton** (`quasi_newton_method`) con los mismos parámetros.

3. **Registro de resultados:** Para cada experimento, guarda:
   - Las coordenadas finales (`x_min`) y el valor mínimo de la función (`f_min`).  
   - Número de iteraciones realizadas y si el método convergió.  
   - Tiempo de ejecución de cada método.

4. **Almacenamiento de resultados:** Todos los resultados se guardan en un archivo JSON llamado `results_<experiment_name>` para su posterior análisis.

In [4]:
def Experiment(experiment_name):
    # Leer configuraciones desde archivo JSON
    with open(experiment_name, "r") as f_in:
        configs = json.load(f_in)

    resultados = []

    for idx, cfg in enumerate(configs):
        # Aplicar Método de Máximo Descenso
        x_min_gd, history_gd, num_iter_gd, converged_gd, elapsed_gd = gradient_descent(
            cfg["x0"], grad_f,
            learning_rate=cfg["learning_rate"],
            tol=cfg["tol"],
            max_iter=cfg["max_iter"]
        )

        # Aplicar Método de Quasi-Newton
        x_min_qn, history_qn, iterations_qn, success_qn, elapsed_qn = quasi_newton_method(
            f, grad_f,
            x0=cfg["x0"],
            tol=cfg["tol"],
            max_iter=cfg["max_iter"]
        )

        #Guardar ambos resultados
        resultados.append({
            "params": cfg,

            # Resultados de Gradiente Descendente
            "gradient_descent": {
                "x_min": x_min_gd.tolist(),
                "f_min": float(f(x_min_gd)),
                "num_iter": num_iter_gd,
                "converged": converged_gd,
                "time_seconds": elapsed_gd
            },

            # Resultados de Quasi-Newton
            "quasi_newton": {
                "x_min": x_min_qn.tolist(),
                "f_min": float(f(x_min_qn)),
                "num_iter": iterations_qn,
                "converged": success_qn,
                "time_seconds": elapsed_qn
            }
        })

    # Guardar resultados en archivo JSON
    with open(f"results_{experiment_name}", "w") as f_out:
        json.dump(resultados, f_out, indent=4)

    print("✅ Experimento completado.")